In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
#figuring out method 1, i might need a pom.xml file; jar to be auto downloaded from maven coordinates
# second config worked as the jar is available on local dir. first method, i think the container doesnt hv access to the internet will investigate.
spark = (
    SparkSession.builder.appName("DataSourcesJDBC")
    .config(
        "spark.jars.packages", "org.postgresql:postgresql:42.7.2"
    )
    .config(
        "spark.driver.extraClassPath", "/pysparkdata/postgresql-42.7.2.jar"
    )
    .getOrCreate()
)

In [2]:
!ls /pysparkdata

LearningSparkV2  postgresql-42.7.2.jar	SparkDefinitive


In [22]:
#method 2 if jar files are locally downloaded
spark = SparkSession.builder.appName("DataSource") \
    .config("spark.jars.packages", "/data/postgresql-42.7.2.jar") \
    .getOrCreate()

In [6]:
'''our DB connection PARAMETERS.'''

host = "learnpyspark-postgres-1" #Name of the container is the host
port = 5432
user = "postgres"
password = "password"
database = "testdb"
dbtbl = "testtable"
url = f"jdbc:postgresql://{host}:{port}/{database}"

In [18]:
#reading from the postgres database
jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", url) \
    .option("dbtable", dbtbl) \
    .option("user", user) \
    .option("password", password) \
    .load()

jdbcDF.printSchema()

root
 |-- id: integer (nullable = true)
 |-- custname: string (nullable = true)
 |-- email: string (nullable = true)
 |-- tel: integer (nullable = true)



In [19]:
jdbcDF.show()

+---+--------+---------------+------+
| id|custname|          email|   tel|
+---+--------+---------------+------+
|  1|   moris|    moris@gmail|  3540|
|  2|    mumo|     mumo@gmail|  5689|
|  3|   masua|    masua@gmail|  6398|
|  4| Charlie|charlie@example|987654|
|  5|   Alice|  alice@example|123456|
|  6|     Bob|    bob@example|654321|
|  7|     vee|      vee@gmail|  5478|
|  8|   daisy|    daisy@gmail|  7778|
+---+--------+---------------+------+



In [9]:
#reading to the postgres database. step1 -load or creat a df, step2 - save to db
data = [
    ("Alice", "alice@example", 123456),
    ("Bob", "bob@example", 654321),
    ("Charlie", "charlie@example", 987654),
]

# Create a DataFrame from the list
df = spark.createDataFrame(data, schema="custname string, email string, tel int")

# Show the DataFrame
df.show()

+--------+---------------+------+
|custname|          email|   tel|
+--------+---------------+------+
|   Alice|  alice@example|123456|
|     Bob|    bob@example|654321|
| Charlie|charlie@example|987654|
+--------+---------------+------+



In [15]:
#USING append mode and the save()
df.write \
    .format("jdbc") \
    .mode('append') \
    .option("url", url) \
    .option("dbtable", dbtbl) \
    .option("user", user) \
    .option("password", password) \
    .save()